In [ ]:

# !pip install foolbox
# import foolbox as fb
from google.colab import files

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime 
from tqdm.notebook import tqdm 
import statistics
from math import log10
import struct
from random import randrange
import multiprocessing
import concurrent.futures
import time

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib 

# parameters
RANDOM_SEED = 42
BATCH_SIZE = 100
# N_EPOCHS = 1575
IMG_SIZE = 32
N_CLASSES = 10

# LEARNING_RATE = 0.001
# MOMENTUM = 0.9
# WEIGHT_DECAY = 1e-3


transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# download and create datasets
train_dataset = datasets.CIFAR10(root='cifar10_data', train=True, transform=transforms, download=True)

valid_dataset = datasets.CIFAR10(root='cifar10_data', train=False, transform=transforms, download=True)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

#Class labels
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')


#implementing AlexNet_Exact model
class AlexNet_Exact(nn.Module):

    def __init__(self):
        super(AlexNet_Exact, self).__init__()        

        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(6, 6))

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(2,2))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=(3,3), stride=(1,1), padding=(2,2))
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
                
        self.dropout = nn.Dropout()
        self.linear1 = nn.Linear(in_features = 256 * 6 * 6, out_features = 1024)
        self.linear2 = nn.Linear(in_features = 1024, out_features = 1024)
        self.linear3 = nn.Linear(in_features = 1024, out_features = 10)
        
        
    def forward(self, x):
        # Feature extractor
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.relu(x)

        x = self.conv5(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.avgpool(x)
        
        x = x.view(x.size(0), 256 * 6 * 6)
        
        x = self.dropout(x)
        x = self.linear1(x)
        x = self.relu(x)
        
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        
        logits = self.linear3(x)
#         probs = F.softmax(logits, dim=1)
        
        return logits



torch.manual_seed(RANDOM_SEED)
target_model = AlexNet_Exact()

model_path = '../dataset/Cifar10.pth'
target_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
target_model.eval()



Files already downloaded and verified
Files already downloaded and verified


AlexNet_Exact(
  (relu): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=9216, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (linear3): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
#Testing Accuracy
def get_accuracy(model, data_loader):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in data_loader:
          images, labels = data[0], data[1]
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = (correct / total)
  return accuracy

valid_acc = get_accuracy(target_model, valid_loader)
print(f'Test_acc:', valid_acc)

valid_acc: 0.8274


In [ ]:
'''
Attacker train
'''
input_mem = []
label_mem = []
label_cifar10_mem = []

with torch.no_grad():
  target_model.eval()
  for i, data in enumerate(train_loader, 0):
    image, label = data[0], data[1]
    #batch size 100; so, member = 50*100 = 5000
    if i<50:
      label_cifar10_mem.append(label)   
      logit = target_model(image)
      input_mem.append(logit)
      label_mem = label_mem + [1 for i in range(BATCH_SIZE)]
    

input_nonmem = []
label_nonmem = []
label_cifar10_nonmem = []


with torch.no_grad():
  target_model.eval()
  for i, data in enumerate(valid_loader, 0):
    image, label = data[0], data[1]
    #batch size 100; so, member = 50*100 = 5000
    if i<50:  
      label_cifar10_nonmem.append(label)  
      logit = target_model(image) #logit is tensor here
      input_nonmem.append(logit)
      label_nonmem = label_nonmem + [0 for i in range(BATCH_SIZE)]


label_cifar10_mem_nonmem = label_cifar10_mem + label_cifar10_nonmem
y_cifar10_train = torch.cat(label_cifar10_mem_nonmem, dim=0)

input_mem_nonmem = input_mem + input_nonmem
X_train_mem_nonmem = torch.cat(input_mem_nonmem, dim=0)
# print(X_train_mem_nonmem.size())
label_mem_nonmem = label_mem + label_nonmem
y_train_attacker_np = np.array(label_mem_nonmem)



In [ ]:
'''
Attacker test
'''

input_mem = []
label_mem = []
label_cifar10_mem = []

with torch.no_grad():
  target_model.eval()
  for i, data in enumerate(train_loader, 0):
    image, label = data[0], data[1]
    #batch size 100; so, member = 50*100 = 5000
    if i>=50 and i<100: 
      label_cifar10_mem.append(label) 
      logit = target_model(image)
      input_mem.append(logit)
      label_mem = label_mem + [1 for i in range(BATCH_SIZE)]
      
    


input_nonmem = []
label_nonmem = []
label_cifar10_nonmem = []

with torch.no_grad():
  target_model.eval()
  for i, data in enumerate(valid_loader, 0):
    image, label = data[0], data[1]
    #batch size 100; so, member = 50*100 = 5000
    if i>=50 and i<100:
      label_cifar10_nonmem.append(label)  
      logit = target_model(image) #logit is tensor here
      input_nonmem.append(logit)
      label_nonmem = label_nonmem + [0 for i in range(BATCH_SIZE)]
    

label_cifar10_mem_nonmem = label_cifar10_mem + label_cifar10_nonmem
y_cifar10_test = torch.cat(label_cifar10_mem_nonmem, dim=0)

input_mem_nonmem = input_mem + input_nonmem
X_test_mem_nonmem = torch.cat(input_mem_nonmem, dim=0)
# print(X_mem_nonmem.size())
y_attacker =  np.array(label_mem_nonmem)
label_mem_nonmem = label_mem + label_nonmem
y_test_attacker_np = np.array(label_mem_nonmem)


In [ ]:
'''
I_nn MIA. logit+prob
'''

logit = X_train_mem_nonmem
df_logit = pd.DataFrame(logit.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
df_logit  = pd.DataFrame(scaler.fit_transform(df_logit))

X_mem_nonmem_prob = torch.sigmoid(logit)
# X_mem_nonmem_prob = torch.softmax(logit, dim=-1)
df_prob = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())
X_train_attacker_df = pd.concat([df_logit, df_prob], axis=1, ignore_index=True)

#=============== Attacker Training ==================
n_feature = len(X_train_attacker_df.columns) 
attacker_mlp_logit_prob = MLPClassifier(hidden_layer_sizes=(n_feature, 100, 100, 50), activation='relu', max_iter = 3000, random_state = 1)
attacker_mlp_logit_prob.fit(X_train_attacker_df, y_train_attacker_np)
X_attacker = pd.concat([df_logit, df_prob], axis=1, ignore_index=True)

# y_pred_np = attacker_mlp_logit_prob.predict(X_train_attacker_df)
# accuracy  = round(np.mean(y_pred_np == y_train_attacker_np), 2)
# print(f'MIA train accuracy: {accuracy}')



model_file = '../dataset/cifar10_attacker_mlp_logit_prob.pkl'
joblib.dump(attacker_mlp_logit_prob, model_file)
attacker_mlp_logit_prob = joblib.load(model_file)

#=============== Attacker Testing ==================
logit = X_test_mem_nonmem
df_logit = pd.DataFrame(logit.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
df_logit  = pd.DataFrame(scaler.fit_transform(df_logit))
X_mem_nonmem_prob = torch.sigmoid(logit)
df_prob = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())
X_test_attacker_df = pd.concat([df_logit, df_prob], axis=1, ignore_index=True)


y_pred_np = attacker_mlp_logit_prob.predict(X_attacker)
accuracy  = round(np.mean(y_pred_np == y_attacker), 2)
print(f'I_nn MIA: {accuracy}')



I_nn MIA: 0.91


In [ ]:
'''
I_bb MIA
'''
df_logit = pd.DataFrame(X_train_mem_nonmem.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
X_train_attacker_df  = pd.DataFrame(scaler.fit_transform(df_logit))


#=============== Attacker Training ==================
n_feature = len(X_train_attacker_df.columns) 
attacker_mlp_logit = MLPClassifier(hidden_layer_sizes=(n_feature, 100, 100, 50), activation='relu', max_iter = 3000, random_state = 1)
attacker_mlp_logit.fit(X_train_attacker_df, y_train_attacker_np)
X_attacker = pd.DataFrame(scaler.fit_transform(df_logit))

# y_pred_np = attacker_mlp_logit.predict(X_train_attacker_df)
# accuracy  = round(np.mean(y_pred_np == y_train_attacker_np), 2)
# print(f'MIA train accuracy: {accuracy}')


model_file = '../dataset/cifar10_attacker_mlp_logit.pkl'
joblib.dump(attacker_mlp_logit, model_file)
attacker_mlp_logit = joblib.load(model_file)

#=============== Attacker Testing ==================
df_logit = pd.DataFrame(X_test_mem_nonmem.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
X_test_attacker_df  = pd.DataFrame(scaler.fit_transform(df_logit))


y_pred_np = attacker_mlp_logit.predict(X_attacker)
accuracy  = round(np.mean(y_pred_np == y_attacker), 2)
print(f'I_bb MIA: {accuracy}')


I_bb MIA: 0.89


In [ ]:
'''
I_bl MIA 
'''

def myCustomLoss(my_outputs, my_labels):
    #specifying the batch size
    my_batch_size = my_outputs.size()[0] 
    #calculating the log of softmax values           
    my_outputs = F.log_softmax(my_outputs, dim=1)  
    #selecting the values that correspond to labels
    my_outputs = my_outputs[range(my_batch_size), my_labels] 
    #returning the results
    return my_outputs


logit = X_train_mem_nonmem
df_logit = pd.DataFrame(logit.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
df_logit  = pd.DataFrame(scaler.fit_transform(df_logit))

X_mem_nonmem_prob = torch.sigmoid(logit)
# X_mem_nonmem_prob = torch.softmax(logit, dim=-1)
df_prob = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

loss = myCustomLoss(logit, y_cifar10_train)
df_loss = pd.DataFrame(loss.detach().numpy())
# scaler = StandardScaler() #94 70
scaler = RobustScaler() #94 70
df_loss  = pd.DataFrame(scaler.fit_transform(df_loss))

X_train_attacker_df = pd.concat([df_logit, df_prob, df_loss], axis=1, ignore_index=True)


#=============== Attacker Training ==================
n_feature = len(X_train_attacker_df.columns) 
attacker_mlp_logit_prob_loss = MLPClassifier(hidden_layer_sizes=(n_feature, 100, 100, 25), activation='relu', max_iter = 3000, random_state = 1)
attacker_mlp_logit_prob_loss.fit(X_train_attacker_df, y_train_attacker_np)
X_attacker = pd.concat([df_logit, df_prob, df_loss], axis=1, ignore_index=True)

# y_pred_np = attacker_mlp_logit_prob_loss.predict(X_train_attacker_df)
# accuracy  = round(np.mean(y_pred_np == y_train_attacker_np), 2)
# print(f'MIA train accuracy: {accuracy}')


model_file = '../dataset/cifar10_attacker_mlp_logit_prob_loss.pkl'
joblib.dump(attacker_mlp_logit_prob_loss, model_file)
attacker_mlp_logit_prob_loss = joblib.load(model_file)


#=============== Attacker Testing ==================
logit = X_test_mem_nonmem
df_logit = pd.DataFrame(logit.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
df_logit  = pd.DataFrame(scaler.fit_transform(df_logit))

X_mem_nonmem_prob = torch.sigmoid(logit)
# X_mem_nonmem_prob = torch.softmax(logit, dim=-1)
df_prob = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

loss = myCustomLoss(logit, y_cifar10_test)
df_loss = pd.DataFrame(loss.detach().numpy())
# scaler = StandardScaler() #94 70
scaler = RobustScaler() #94 70
df_loss  = pd.DataFrame(scaler.fit_transform(df_loss))

X_test_attacker_df = pd.concat([df_logit, df_prob, df_loss], axis=1, ignore_index=True)

# y_pred_np = attacker_mlp_logit_prob_loss.predict(X_test_attacker_df)
# accuracy  = round(np.mean(y_pred_np == y_test_attacker_np), 2)
# print(f'MIA test accuracy: {accuracy}')

y_pred_np = attacker_mlp_logit_prob_loss.predict(X_attacker)
accuracy  = round(np.mean(y_pred_np == y_attacker), 2)
print(f'I_bl MIA: {accuracy}')


I_bl MIA: 0.91
